<img src="./Img/Label_02.png">

# Зачет

28 мая 2021 года <br>
ПИ19-3, ПИ19-4 - 3 подгруппа<br>

4 июня 2021 года <br>
ПИ19-4, ПИ19-5 - 4 подгруппа

5 июня 2021 года <br>
ПИ19-2, ПИ19-3, ПИ19-4 - 2 подгруппа<br>

In [13]:
#6 orm

"""
Напишите запрос для получения списка названий университетов вместе с
названиями преподаваемых в них предметов.
"""


'\nНапишите запрос для получения списка названий университетов вместе с\nназваниями преподаваемых в них предметов.\n'

In [14]:
# Импортим дофига разных модулей для работы sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from datetime import datetime

# Подключения к базе данных
engine = create_engine('sqlite:///Students_2021.sqlite')
Base = automap_base()
Base.prepare(engine, reflect=True)
# Создание текущей сессии
session = Session(engine)

# Смотрим, какие есть таблицы в бд и передаем их в отдельные переменные (впринципе print можно и стереть)
print(Base.classes.values())
city, exam_marks, student, university, subject, lecturer, subj_lect = Base.classes.values()

[<class 'sqlalchemy.ext.automap.city'>, <class 'sqlalchemy.ext.automap.exam_marks'>, <class 'sqlalchemy.ext.automap.student'>, <class 'sqlalchemy.ext.automap.university'>, <class 'sqlalchemy.ext.automap.subject'>, <class 'sqlalchemy.ext.automap.lecturer'>, <class 'sqlalchemy.ext.automap.subj_lect'>]


In [15]:
print('Университет\tПредмет')

# Это сам select, какие выбираем колонки
query = session.query(university.univ_name, subject.subj_name)
# Соединяем таблицы university и lecturer
.join(lecturer, university.univ_id == lecturer.univ_id)
# Потом еще subj_lect к тем двум сверху
.join(subj_lect, lecturer.lecturer_id == subj_lect.lecturer_id)
# И еще subject туда же
.join(subject, subj_lect.subj_id == subject.subj_id)
# Сортировка по названию универа
.order_by(university.univ_name)
# запрос всех строк
.all()

# Получается структура [(), ()], поэтому выводим "построчно"
for item in query:
    print(item)

Университет	Предмет
('БГУ', 'Анализ данных')
('БГУ', 'Базы данных')
('БГУ', 'Программирование на языках Python и SQL')
('ВГМА', 'Анализ данных')
('ВГМА', 'Математика')
('ВГУ', 'История')
('МГТУ', 'Физкультура')
('МГТУ', 'Финансовые риски корпоративного инвестирования')
('МГУ', 'История')
('МТУСИ', 'Базы данных')
('МТУСИ', 'Экономическая информатика')
('НГУ', 'Физика')
('Политех', 'Информатика')
('Политех', 'Программирование')
('Политех', 'Web-дизайн')
('РГУ', 'Финансовые рынки')


In [ ]:
#7 core

"""
Напишите запрос для получения списка университетов с указанием
суммарного количества аудиторных часов в каждом семестре.
"""


In [16]:
# Опять много импортов
from sqlalchemy import (select, create_engine, MetaData, Table, Numeric, String, 
                        insert, update, delete, cast)
from datetime import datetime
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [17]:
# Подключение к бд
engine=create_engine('sqlite:///Students_2021.sqlite')
metadata=MetaData()
connection=engine.connect()

In [18]:
# передаем таблицы из бд в отдельные переменные
city=Table('city', metadata, autoload=True, autoload_with=engine)
exam_marks=Table('exam_marks', metadata, autoload=True, autoload_with=engine)
student=Table('student', metadata, autoload=True, autoload_with=engine)
university=Table('university', metadata, autoload=True, autoload_with=engine)
subject=Table('subject', metadata, autoload=True, autoload_with=engine)
lecturer=Table('lecturer', metadata, autoload=True, autoload_with=engine)
subj_lect=Table('subj_lect', metadata, autoload=True, autoload_with=engine)

In [28]:
print('Университет, Семестр, Часы')

# Сам select
selection=select([university.c.univ_name, subject.c.semester, func.sum(subject.c.hour)])
# Указываем, откуда берем данные
.select_from(
    # Длинная цепочка joinтов
    university.join(lecturer).join(subj_lect).join(subject)
    )
# группировка по семестру и университету
.group_by(subject.c.semester, university.c.univ_name)
# сортировка по университету
.order_by(university.c.univ_name)

# Отправляем сформированный select в бд и дергаем ответ
result=connection.execute(selection).fetchall()

# ну и вывод
for item in result:
    print(item)

Университет, Семестр, Часы
('БГУ', 1, 42)
('БГУ', 7, 360)
('ВГМА', 1, 42)
('ВГМА', 2, 56)
('ВГУ', 4, 34)
('МГТУ', 5, 70)
('МГУ', 4, 34)
('МТУСИ', 7, 252)
('НГУ', 1, 34)
('Политех', 1, 56)
('Политех', 2, 34)
('Политех', 4, 72)
('РГУ', 6, 72)
